## Small Web-Scrapping Temperature Project

In [19]:
import requests
import pandas as pd

In [28]:
# Library for parsing HTML
from bs4 import BeautifulSoup

In [11]:
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]


<bound method Tag.prettify of <div class="tombstone-container">
<p class="period-name">Tonight<br/><br/></p>
<p><img alt="Tonight: Mostly cloudy, with a low around 54. Breezy, with a west wind 16 to 23 mph, with gusts as high as 31 mph. " class="forecast-icon" src="DualImage.php?i=nwind_bkn&amp;j=nbkn" title="Tonight: Mostly cloudy, with a low around 54. Breezy, with a west wind 16 to 23 mph, with gusts as high as 31 mph. "/></p><p class="short-desc">Mostly Cloudy<br/>and Breezy<br/>then Mostly<br/>Cloudy</p><p class="temp temp-low">Low: 54 °F</p></div>>


In [12]:
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Tonight: Mostly cloudy, with a low around 54. Breezy, with a west wind 16 to 23 mph, with gusts as high as 31 mph. " class="forecast-icon" src="DualImage.php?i=nwind_bkn&amp;j=nbkn" title="Tonight: Mostly cloudy, with a low around 54. Breezy, with a west wind 16 to 23 mph, with gusts as high as 31 mph. "/>
 </p>
 <p class="short-desc">
  Mostly Cloudy
  <br/>
  and Breezy
  <br/>
  then Mostly
  <br/>
  Cloudy
 </p>
 <p class="temp temp-low">
  Low: 54 °F
 </p>
</div>


In [13]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

Tonight
Mostly Cloudyand Breezythen MostlyCloudy
Low: 54 °F


In [14]:
img = tonight.find("img")
desc = img['title']
print(desc)

Tonight: Mostly cloudy, with a low around 54. Breezy, with a west wind 16 to 23 mph, with gusts as high as 31 mph. 


In [15]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Tonight',
 'Friday',
 'FridayNight',
 'Saturday',
 'SaturdayNight',
 'Sunday',
 'SundayNight',
 'Monday',
 'MondayNight']

In [18]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['Mostly Cloudyand Breezythen MostlyCloudy', 'Mostly Cloudy', 'Mostly Cloudy', 'Mostly Cloudy', 'Chance Rain', 'Rain Likelythen ChanceRain', 'Mostly Cloudy', 'Mostly Sunny', 'Mostly Clear']
['Low: 54 °F', 'High: 64 °F', 'Low: 53 °F', 'High: 64 °F', 'Low: 57 °F', 'High: 65 °F', 'Low: 54 °F', 'High: 67 °F', 'Low: 51 °F']
['Tonight: Mostly cloudy, with a low around 54. Breezy, with a west wind 16 to 23 mph, with gusts as high as 31 mph. ', 'Friday: Mostly cloudy, with a high near 64. West wind 13 to 21 mph, with gusts as high as 26 mph. ', 'Friday Night: Mostly cloudy, with a low around 53. West southwest wind 11 to 20 mph, with gusts as high as 25 mph. ', 'Saturday: Mostly cloudy, with a high near 64. West southwest wind 7 to 17 mph, with gusts as high as 22 mph. ', 'Saturday Night: A 50 percent chance of rain, mainly after 11pm.  Mostly cloudy, with a low around 57. West southwest wind 7 to 14 mph, with gusts as high as 18 mph.  New precipitation amounts of less than a tenth of an inch 

In [20]:
weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc": descs
})
weather

,period,short_desc,temp,desc
0,Tonight,Mostly Cloudyand Breezythen MostlyCloudy,Low: 54 °F,"Tonight: Mostly cloudy, with a low around 54. ..."
1,Friday,Mostly Cloudy,High: 64 °F,"Friday: Mostly cloudy, with a high near 64. We..."
2,FridayNight,Mostly Cloudy,Low: 53 °F,"Friday Night: Mostly cloudy, with a low around..."
3,Saturday,Mostly Cloudy,High: 64 °F,"Saturday: Mostly cloudy, with a high near 64. ..."
4,SaturdayNight,Chance Rain,Low: 57 °F,"Saturday Night: A 50 percent chance of rain, m..."
5,Sunday,Rain Likelythen ChanceRain,High: 65 °F,"Sunday: Rain likely, mainly before 11am. Most..."
6,SundayNight,Mostly Cloudy,Low: 54 °F,"Sunday Night: Mostly cloudy, with a low around..."
7,Monday,Mostly Sunny,High: 67 °F,"Monday: Mostly sunny, with a high near 67."
8,MondayNight,Mostly Clear,Low: 51 °F,"Monday Night: Mostly clear, with a low around 51."


In [23]:
temp_nums = weather["temp"].str.extract(r'(?P<temp_num>\d+)', expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    54
1    64
2    53
3    64
4    57
5    65
6    54
7    67
8    51
Name: temp_num, dtype: object

In [24]:
weather["temp_num"].mean()

58.77777777777778

In [25]:
is_night = weather["temp"].str.contains("low")
weather["is_night"] = is_night
is_night

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
Name: temp, dtype: bool

In [27]:
weather["is_night"]

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
Name: is_night, dtype: bool